## Carregar base de Dados

In [ ]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Seed para reprodução de resultados
seed = 1
random.seed(seed)
np.random.seed(seed)

In [ ]:
data = pd.read_excel('basetccfinal.xlsx')
data.head()


In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
n = data.nunique(axis=0)
print("No.of.unique values in each column :\n",
      n)

In [ ]:
data2=data.copy()
data2=data2.drop(columns=['preco','Total Impostos (ICMS e FCP)','Index','Cod Familia Item','Dsc 1 Item','Business Unit Cd', 'Business Unit Desc','CNPJ / CPF','Cod Comprador Contrato','Cod Comprador Ped Atual','Cod Filial / Fábrica','Depto Comprador Contrato','Dsc 2 Item','Dsc Familia Item','Dsc Filial / Fábrica','Nom Comprador Ped Atual','Nom Comprador Ped Atual','Nome Comprador Contrato','Número Contrato','Servico (S/N)','Subconta','Unidade Medida Compra','Área Suprimentos Atual','Área Suprimentos Original','Área Suprimentos','dolarizar','Dsc Tipo Pedido','Descrição Condição Pagamento','Nome Conta','Tipo Conta','Mes Emissão NF','Mês Criação Pedido','Generico'])
data2 = data2.loc[:, ~data2.columns.str.contains('^Unnamed')]

In [ ]:
data2.head()

In [ ]:
data2.shape


In [ ]:
col = data2.select_dtypes(exclude=['number']).columns.tolist()
for col in data2.columns:
    unique_values = set(data2[col].apply(type))
    print(col, unique_values)

In [ ]:
# converter todas as colunas numéricas para float
num_cols = data2.select_dtypes(include=[int, float]).columns
data2.loc[:, num_cols] = data2.loc[:, num_cols].astype('float').copy()

# converter todas as colunas de texto para string
text_cols = data2.select_dtypes(include=object).columns
data2.loc[:, text_cols] = data2.loc[:, text_cols].astype('object').copy()

In [ ]:

for col in data2.columns:
    unique_values = set(data2[col].apply(type))
    print(col, unique_values)

In [ ]:
# Identificar as colunas datetime64
cols_datetime = data2.select_dtypes(include=['datetime64']).columns.tolist()

# Imprimir as colunas identificadas
print(cols_datetime)

# Transformação de variáveis categórias em númericas usando LabelEncoder

In [ ]:
data2['data_fechamento'] = pd.to_datetime(data2['data_fechamento'])

In [ ]:
data2['data_fechamento'].head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load

# Separar as variáveis numéricas das categóricas
Colunas_num = data2.select_dtypes(include=['float', 'int']).columns.tolist()
X_numerical = data2[Colunas_num]

# Criar um dicionário para armazenar os objetos LabelEncoder
label_encoders = {}
data_transf= pd.DataFrame()
colunas_obj = data2.select_dtypes(include = "object").columns

# Iterar sobre as colunas categóricas do dataframe
for obj in colunas_obj:
    # Criar um novo objeto LabelEncoder
    le = LabelEncoder()
    # Aplicar o LabelEncoder na coluna categórica
    data_transf[obj] = le.fit_transform(data2[obj].astype(str))
    # Salvar o objeto LabelEncoder em um arquivo
    dump(le, f'{obj}_label_encoder.joblib')
    # Armazenar o objeto LabelEncoder no dicionário
    label_encoders[obj] = le

# Converter a coluna datetime64 para segundos desde o Unix Epoch
unix_time = data2['data_fechamento'].min()
X_date = (data2['data_fechamento'] - unix_time).dt.total_seconds()

# Criação do novo dataframe
df_tratado = pd.concat([X_numerical, data_transf, X_date], axis=1)
df_tratado.head()


In [ ]:
print(df_tratado.isna().sum())

In [ ]:
for col in df_tratado.columns:
    unique_values = set(df_tratado[col].apply(type))
    print(col, unique_values)

# Tratar missing values


In [ ]:
import missingno as msno
X = df_tratado.drop('cons_preco', axis=1)
msno.matrix(X, figsize=(25,5));

In [ ]:
print(df_tratado.isna().sum())

In [ ]:
from sklearn.impute import SimpleImputer

# Cria um objeto SimpleImputer
imputer = SimpleImputer(strategy='most_frequent', fill_value=0)
X = df_tratado

# Aplica o imputador aos dados de entrada
X_tratado = imputer.fit_transform(X)

data_SVM = pd.DataFrame(X_tratado, columns=df_tratado.columns)

In [ ]:
print(data_SVM.isna().sum())

In [ ]:
data_SVM.head()

# Pré-Processamento para aplicar o PCA

In [ ]:
from sklearn.decomposition import PCA
data_pca=data_SVM
data_pca.head()

In [ ]:
print(data_pca.columns)

In [ ]:
#Vamos aplicar o PCA para buscar melhorar o modelo, avaliando o impacto desta etapa nos esultados do modelo.
#Pré-processamento PCA (padronização e normalização). 
#Como o PCA é sensível à escala dos dados, vamos usar o fit_transform para ajustar a escala dos dados com base na distribuição e transformar os dados originais para que possam ser utilizados pelo PCA.
# Vamos também utilizar o StandardScaler para padronizar os recursos do conjunto de dados (Média 0 e desvio 1)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

y = data_pca['cons_preco']
X_numerical = data_pca.drop('cons_preco', axis=1)

# Aplicar normalização nos dados numéricos
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_numerical)

# Juntar a variável target novamente com os dados normalizados
data_normalized = pd.concat([pd.DataFrame(X_normalized , columns=X_numerical.columns), y], axis=1)

print(data_normalized.columns)



In [ ]:
# Plota as densidades de probabilidade antes e depois da normalização
for col in num_cols:
    fig, ax = plt.subplots(ncols=2, figsize=(16, 4))

    # Plot da densidade de probabilidade antes da normalização
    sns.kdeplot(data_pca[col], ax=ax[0])
    ax[0].set_title(f"{col} antes da normalização")

    # Plot da densidade de probabilidade depois da normalização
    sns.kdeplot(data_normalized[col], ax=ax[1])
    ax[1].set_title(f"{col} depois da normalização")

    plt.show()


In [ ]:
data_pca2 = data_normalized

# Aplicando o PCA

In [ ]:
#DIvidindo as bases de treino e de teste

from sklearn.model_selection import train_test_split

data_rf_pca = data_pca2

# Separar os dados em variáveis preditoras (X) e target (y)
X = data_rf_pca.drop('cons_preco', axis=1)
y = data_rf_pca['cons_preco']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#balanceamento da base de treino

from imblearn.over_sampling import RandomOverSampler

# Cria o objeto da classe RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Aplica o resampling no conjunto de treino
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

# Imprime o número de exemplos em cada conjunto
print("Número de exemplos no conjunto de treino antes do balanceamento: ", X_train.shape[0])
print("Número de exemplos no conjunto de treino depois do balanceamento: ", X_train_res.shape[0])


In [ ]:
#rodando o PCA

from sklearn.decomposition import PCA


# Cria um objeto PCA com n_components=0.95 para manter 95% da variância
pca = PCA(n_components=0.95)

# Aplica o PCA nos dados X e armazena em X_pca
X_pca_res = pca.fit_transform(X_train_res)


# Criando o novo dataframe com as novas features geradas pelo PCA
data_pca_res = pd.concat([pd.DataFrame(X_pca_res), y_train_res], axis=1)

## RandomForest pós PCA sem class weight

In [ ]:
def predict_and_evaluate(X_test, y_test, model, average='macro'):

  # inferência do teste
  y_pred = model.predict(X_test) 

  # Acurácia
  from sklearn.metrics import accuracy_score
  accuracy = accuracy_score(y_test, y_pred)
  print('Acurácia: ', accuracy)

  # Kappa
  from sklearn.metrics import cohen_kappa_score
  kappa = cohen_kappa_score(y_test, y_pred)
  print('Kappa: ', kappa)

  # F1
  from sklearn.metrics import f1_score
  f1 = f1_score(y_test, y_pred, average=average)
  print('F1: ', f1)

  from sklearn.metrics import precision_score, recall_score, roc_auc_score

  # Precisão
  precision = precision_score(y_test, y_pred, average='macro')
  print('Precisão: ', precision)

  # Recall
  recall = recall_score(y_test, y_pred, average='macro')
  print('Recall: ', recall)

  # Matriz de confusão
  from sklearn.metrics import confusion_matrix
  # Gerando a matriz de confusão
  confMatrix = confusion_matrix(y_test, y_pred)

  # Normalizando a matriz de confusão
  row_sums = confMatrix.sum(axis=1, keepdims=True)
  norm_confMatrix = confMatrix / row_sums

  ax = plt.subplot()
  sns.heatmap(norm_confMatrix, annot=True, fmt=".2f", cmap='Blues')
  ax.set_xlabel('Previsto')
  ax.set_ylabel('Real')
  ax.set_title('Matriz de Confusão Normalizada')

  # Colocar os nomes
  ax.xaxis.set_ticklabels(['0', '1', '2', '3', '4']) 
  ax.yaxis.set_ticklabels(['0', '1', '2', '3', '4'])
  plt.show()

  # Retornar as métricas
  return accuracy, kappa, f1, norm_confMatrix



In [ ]:
data_RFE=data_pca_res

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# separar as características e o target
X = data_RFE.drop('cons_preco', axis=1)
y = data_RFE['cons_preco']

# dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instanciar o modelo
rf = RandomForestClassifier(random_state=42)

# treinar o modelo
rf.fit(X_train, y_train)

# fazer as previsões com o modelo treinado
y_pred = rf.predict(X_test)

# avaliar o modelo
print(classification_report(y_test, y_pred))
average = 'macro'

In [ ]:
predict_and_evaluate(X_test, y_test, rf)